In [0]:
!pip install kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ningom","key":"53b8540e06ccbede8759befaff246b35"}'}

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!mv kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [422]:
!kaggle competitions download -c titanic

gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
#get datas
import pandas as pd
import numpy as np
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
passenger_data=test_df["PassengerId"].values

In [0]:
import re
from pandas import Series,DataFrame
def data_shape(df,data_num,age_index):
  #name
  sep_forward = df["Name"].str.split(",",expand=True)
  sep_backward = sep_forward[1].str.split(".",expand=True)
  print(sep_backward[0].value_counts())
  job = sep_backward[0].str.replace(" ","")
  df["Title"] = job
  df['Title'] =df['Title'].replace("Dr", 'Master')
  df['Title'] =df['Title'].replace(['Don','Rev', 'Major','Col','Capt','Jonkheer'], 'Mr')
  df['Title'] =df['Title'].replace(['Mme','Ms','Lady','Sir','Mlle','theCountess',"Dona"], 'Mrs')
  title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4}
  df['Title'] = df['Title'].map(title_mapping)
  df.drop('Name', axis=1, inplace=True)
  #FamilySize
  df['FamilySize'] = 1+ df['Parch'] + df['SibSp'] 
  #Embarked
  df.Embarked = df.Embarked.replace(['C', 'S', 'Q'], [0, 1, 2])
  df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)
  #Sex
  df.Sex=df.Sex.replace(["male","female"],[0,1])
  #Fare
  df.drop('Fare', axis=1, inplace=True)
  #Ticket
  for index,value in enumerate(df["Ticket"]):
    value=re.sub("\\D", "", value)
    if value !="":
      df["Ticket"][index]=float(value[0])
    elif value=="LINE":
      df["Ticket"][index]="nan"
    else:
      df["Ticket"][index]="nan" 
  df['Ticket'] =df['Ticket'].replace(["nan","5","6","7","8"], 4)
  df['Ticket'] =df['Ticket'].replace("9", 2)

  #Age(単代入法)
  from sklearn import linear_model
  train_rmv_cabin_df=df.drop('Cabin', axis=1)
  train_rmv_age_df=train_rmv_cabin_df.drop('Age', axis=1)
  #Age除去
  rmv_age=train_rmv_age_df.values
  rmv_age=np.delete(rmv_age,0,axis=1)
  #Age-nan除去
  train_rmv_age_nan_df=train_rmv_cabin_df.dropna(how="any")
  train_rmv_age_nan=train_rmv_age_nan_df.values
  model = linear_model.LinearRegression() 
  for index,age in enumerate(df["Age"]):
    if np.isnan(age):#ageにnanがあると
      data=np.zeros((data_num,df.shape[1]-1))
      for i in range(data_num):#300個のデータをランダムでとってくる
        data[i]=train_rmv_age_nan[np.random.randint(0,len(train_rmv_age_nan))]
      correct_data=data[:,age_index].reshape(data_num,1)#正解データ(age),説明変数(それ以外)
      training_data=np.delete(data,[0,age_index],axis=1)
      model.fit(training_data, correct_data)
      if model.predict(rmv_age[index].reshape(1,-1))<0:
        df["Age"][index]=0
      else:
        df["Age"][index]=model.predict(rmv_age[index].reshape(1,-1))
  df["Age"]=df["Age"].astype(int)

  #Cabin
  df.Cabin = df.Cabin.fillna('Unknown')
  levels = []
  # 1文字目を取り出す
  for level in df.Cabin:
    levels.append(level[0])    
  df.Cabin = DataFrame(levels)
  df.Cabin = df.Cabin.replace('T','U')
  df.Cabin = df.Cabin.replace(['A','B','C','D','E','F','G','U'], [0, 1, 2, 3, 4, 5, 6, 7])
  return df

In [425]:
train_df=data_shape(train_df,data_num=300,age_index=4)
test_df=data_shape(test_df,data_num=200,age_index=3)

 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Col               2
 Mlle              2
 Major             2
 Sir               1
 Capt              1
 Lady              1
 Don               1
 Mme               1
 the Countess      1
 Jonkheer          1
 Ms                1
Name: 0, dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be s

 Mr        240
 Miss       78
 Mrs        72
 Master     21
 Col         2
 Rev         2
 Dona        1
 Ms          1
 Dr          1
Name: 0, dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be s

In [0]:
#データの分割(t_train:正解ラベル,x_train:トレーニングデータ)
t_train=train_df["Survived"].values
x_train=train_df
x_train.drop("Survived",axis=1,inplace=True)
x_train=x_train.values

In [0]:
#データの分割
x_test=test_df.values

In [0]:
#正規化
from sklearn.preprocessing import StandardScaler,MinMaxScaler
std_scl = StandardScaler() #MinMaxScaler() or Robustscaler
x_train=std_scl.fit_transform(x_train).astype(np.int64)
x_test=std_scl.fit_transform(x_test).astype(np.int64)

In [0]:
from sklearn.model_selection import train_test_split
#データ分割
X_train,X_valid,Y_train,Y_valid=train_test_split(x_train,t_train,random_state=0)

In [430]:
#モデルの構築(重回帰分析)
#ロジ
from sklearn import linear_model
c = 1.0
model_logistic=linear_model.LogisticRegression(C=c, solver='liblinear', random_state=0)
model_logistic.fit(X_train, Y_train) 

#ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
model_clf = RandomForestClassifier(random_state=0)
model_clf.fit(X_train,Y_train)

#サポベク
from sklearn.svm import SVC
model_svc=SVC()
model_svc.fit(X_train,Y_train)

#NN
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,BatchNormalization,Flatten
from keras.optimizers import Adam

batch_size=50
epochs=25

model_nn = Sequential([                       
    Dense(100,input_dim=train_df.shape[1], activation='relu'),
    BatchNormalization(),
    Dense(50, activation='relu'),
    BatchNormalization(),
    Dropout(0.1),
    Dense(25, activation='relu'),
    BatchNormalization(),
    Dense(12, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_nn.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

history=model_nn.fit(X_train, Y_train, epochs=epochs,batch_size=batch_size)

# gradient boost
from sklearn.ensemble import GradientBoostingClassifier 
model_gb=GradientBoostingClassifier(random_state=1)
model_gb.fit(X_train,Y_train)

# XGboost
!pip install xgboost
from xgboost import XGBClassifier
model_xgb = XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=100)
model_xgb.fit(X_train,Y_train)


Epoch 1/25
668/668 [==============================] - 3s 4ms/step - loss: 0.9217 - acc: 0.5599
Epoch 2/25
668/668 [==============================] - 0s 289us/step - loss: 0.6414 - acc: 0.6677
Epoch 3/25
668/668 [==============================] - 0s 342us/step - loss: 0.6085 - acc: 0.6901
Epoch 4/25
668/668 [==============================] - 0s 275us/step - loss: 0.5871 - acc: 0.7081
Epoch 5/25
668/668 [==============================] - 0s 291us/step - loss: 0.5528 - acc: 0.7290
Epoch 6/25
668/668 [==============================] - 0s 284us/step - loss: 0.5672 - acc: 0.7201
Epoch 7/25
668/668 [==============================] - 0s 275us/step - loss: 0.4803 - acc: 0.7784
Epoch 8/25
668/668 [==============================] - 0s 225us/step - loss: 0.4961 - acc: 0.7844
Epoch 9/25
668/668 [==============================] - 0s 236us/step - loss: 0.5143 - acc: 0.7545
Epoch 10/25
668/668 [==============================] - 0s 284us/step - loss: 0.5272 - acc: 0.7635
Epoch 11/25
668/668 [==========

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [431]:
from sklearn import metrics
predicted_clf = model_clf.predict(x_train)
predicted_svc = model_svc.predict(x_train)
train_loss, train_acc = model_nn.evaluate(x_train, t_train)
predicted_gb = model_gb.predict(x_train)
expected = t_train
print("Model Accuracy")
print("Random  :%f"% metrics.accuracy_score(expected,predicted_clf))
print("SVC     :%f"% metrics.accuracy_score(expected,predicted_svc))
print("NN      :%f"% train_acc )
print("Gradient:%f"% metrics.accuracy_score(expected,predicted_gb))

891/891 [==============================] - 1s 981us/step
Model Accuracy
Random  :0.877666
SVC     :0.835017
NN      :0.856341
Gradient:0.858586


In [0]:
# base predicts
base_pred_2 = model_clf.predict(X_train)
base_pred_3 = model_svc.predict(X_train)
base_pred_4 = np.round(model_nn.predict(X_train))
base_pred_5 = model_gb.predict(X_train)

In [0]:
# stack base predicts for training meta model
stacked_predictions = np.column_stack((base_pred_2, base_pred_3,base_pred_4,base_pred_5))

In [0]:
# train meta model 
meta_model = linear_model.LinearRegression()
meta_model.fit(stacked_predictions, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# final result 
valid_pred_2 = model_clf.predict(X_valid)
valid_pred_3 = model_svc.predict(X_valid)
valid_pred_4 = np.round(model_nn.predict(X_valid))
valid_pred_5 = model_gb.predict(X_valid)
stacked_valid_predictions = np.column_stack((valid_pred_2, valid_pred_3,valid_pred_4,valid_pred_5))

meta_valid_pred = np.round(meta_model.predict(stacked_valid_predictions))



In [0]:
stacked_valid_predictions
df=pd.DataFrame(stacked_valid_predictions)
df_corr = df.corr()
print(df_corr)

          0         1         2         3
0  1.000000  0.847239  0.870081  0.828900
1  0.847239  1.000000  0.872993  0.847239
2  0.870081  0.872993  1.000000  0.832508
3  0.828900  0.847239  0.832508  1.000000


In [0]:
#モデル評価
from sklearn.metrics import accuracy_score
acc_train=accuracy_score(meta_valid_pred,Y_valid)
print(acc_train)

0.8161434977578476


In [0]:
#予測

test_pred_2 = model_clf.predict(x_test)
test_pred_3 = model_svc.predict(x_test)
test_pred_4 = np.round(model_nn.predict(x_test))
test_pred_5 = model_gb.predict(x_test)

stacked_test_predictions = np.column_stack((test_pred_2, test_pred_3,test_pred_4,test_pred_5))

y_test=np.round(meta_model.predict(stacked_test_predictions))

submit_df=pd.DataFrame(passenger_data.T,columns=["PassengerId"])
submit_df["Survived"]=y_test.T

In [0]:
#csvへ
submit_df_int=submit_df.astype(np.int64)
submit_df_int.to_csv("result.csv",index=False)

In [0]:
submit_df_int

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [0]:
!kaggle competitions submit -c titanic -f result.csv -m 'second submit'

100% 2.77k/2.77k [00:00<00:00, 9.05kB/s]
Successfully submitted to Titanic: Machine Learning from Disaster